# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [3]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [4]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [6]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [7]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [8]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [11]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [12]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch**: Start on your hands and knees, alternate between arching your back up (like a cat) and letting it sag down (like a cow). Do 10-15 repetitions.\n- **Bird Dog**: From hands and knees, extend the opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds, then switch sides. Aim for 10 repetitions per side.\n- **Pelvic Tilts**: Lie on your back with knees bent. Flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n  \nThese exercises are gentle and focus on stretching and strengthening the muscles that support your lower back. Be sure to perform them within your comfort level and consult a healthcare professional if you have ongoing or severe back pain.'

In [13]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in maintaining overall health. It supports physical recovery by helping the body repair tissues and regenerate cells. Adequate sleep is essential for mental well-being, as it aids in consolidating memories and supports cognitive functions like learning. Sleep also regulates hormones that influence growth and appetite, contributing to a healthy metabolism. Furthermore, quality sleep strengthens the immune system, making the body better equipped to fight off illnesses. Inadequate or poor-quality sleep can lead to health issues such as impaired immune function, cognitive difficulties, and an increased risk of chronic conditions. Therefore, practicing good sleep hygiene and ensuring sufficient, restful sleep are crucial for overall health and well-being.'

In [14]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- For headaches:\n  - Drinking water to stay hydrated\n  - Applying cold or warm compresses to the head or neck\n  - Resting in a dark, quiet room\n  - Gently massaging temples and neck\n  - Using peppermint or lavender essential oils\n  - Maintaining a regular sleep schedule\n  - Consuming small amounts of caffeine (with caution)\n\n- For stress relief:\n  - Practicing deep breathing exercises\n  - Using progressive muscle relaxation\n  - Engaging in grounding techniques (e.g., naming things you see, hear, feel)\n  - Taking short walks, especially in nature\n  - Listening to calming music\n\nThese methods can help manage stress and headaches naturally and effectively.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [15]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [16]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [17]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch**: Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n\n- **Bird Dog**: From hands and knees, extend opposite arm and leg while engaging your core. Hold each extension for about 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Pelvic Tilts**: Lie on your back with knees bent, flatten your lower back against the floor by tightening your abdominal muscles and tilting your pelvis slightly upward. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfort and may prevent future episodes.'

In [18]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly impacts overall health. Getting enough quality sleep—typically 7-9 hours per night—allows the body to go through proper sleep cycles, including deep sleep and REM sleep, which are essential for physical repair, memory consolidation, and mental health. Maintaining a consistent sleep schedule and creating an optimal sleep environment (such as a cool, dark, quiet, and comfortable space) can improve sleep quality. Poor sleep or insomnia can lead to issues such as weakened immunity, impaired cognitive function, mood disturbances, and increased risk for chronic conditions. Therefore, prioritizing good sleep practices is crucial for overall wellness.'

In [19]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include relaxation techniques such as deep breathing exercises and meditation, herbal teas like chamomile or valerian root, and practicing stress management strategies to reduce overall tension. Additionally, staying well-hydrated and ensuring adequate sleep can help prevent headaches related to stress.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:

An example query where BM25 is better than embeddings is: “What is CBT-I?”

BM25 would perform better in this case because it relies on exact keyword matching and gives higher weight to specific and rare terms like “CBT-I.” Since this is a precise acronym, exact matching is crucial. Embedding-based retrieval might struggle with rare or domain-specific acronyms if they are not well represented in the training data, which can lead to retrieving more general content about therapy or insomnia. In contrast, BM25 is more likely to retrieve the document that explicitly contains the exact term “CBT-I,” resulting in higher precision.

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [20]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [23]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'According to the information provided, exercises that can help with lower back pain include:\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back up (cat) and letting it sag down (cow). Repeat 10-15 times.\n- **Bird Dog:** From hands and knees, extend the opposite arm and leg while engaging your core. Hold each extension for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, tighten your abs, and tilt your pelvis slightly to flatten your back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfort and prevent future episodes.'

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep greatly affects overall health by supporting physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate sleep—typically 7-9 hours per night—is essential for maintaining good health and well-being.'

In [26]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include drinking water to stay hydrated, applying cold or warm compresses to the head or neck, resting in a dark and quiet room, gently massaging the temples and neck, using peppermint or lavender essential oils, and maintaining a regular sleep schedule. Additionally, practicing deep breathing, progressive muscle relaxation, grounding techniques, taking short walks in nature, and listening to calming music can help reduce stress and alleviate headaches.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [27]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [28]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [29]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nRemember to perform these exercises gently and consult with a healthcare pr

In [30]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical repair, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adequate, quality sleep—typically 7-9 hours per night—supports a strong immune system, helps manage stress, and promotes emotional regulation. Poor sleep or sleep disorders like insomnia can lead to various health issues, including increased stress, fatigue, and a weakened immune response. Therefore, maintaining good sleep hygiene and creating an optimal sleep environment are crucial for sustaining overall health.'

In [31]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using essential oils like peppermint or lavender\n- Maintaining a regular sleep schedule\n- Practicing relaxation techniques such as deep breathing, progressive muscle relaxation, or grounding techniques\n- Taking short walks, preferably in nature\n- Listening to calming music\n\nThese methods can help alleviate headache symptoms and reduce stress naturally.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

Generating multiple reformulations of a user query improves recall because different documents may express the same concept using different words or phrasing. A single query might miss relevant results if the wording does not match the terms used in the documents. By creating several paraphrased or semantically varied versions of the query, the retriever increases the chances of matching more relevant documents. The results retrieved for each reformulation are then combined (union), meaning that documents found through different query variations are gathered together, which increases the total number of relevant documents in the context. This approach helps overcome the lexical gap between the user’s wording and the document vocabulary, thereby increasing overall recall.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [32]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [33]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [34]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [35]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [36]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [37]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Based on the provided information, exercises that can help with lower back pain include:\n\n1. **Cat-Cow Stretch**: On hands and knees, alternate arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n2. **Bird Dog**: From hands and knees, extend opposite arm and leg while engaging your core. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n3. **Partial Crunches**: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and lift your shoulders off the floor. Do 8-12 repetitions.\n4. **Knee-to-Chest Stretch**: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n5. **Pelvic Tilts**: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate lower back discomfor

In [39]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep positively affects overall health in several ways. It plays a crucial role in physical recovery, as during sleep the body repairs tissues and regenerates cells. Sleep also supports mental well-being by consolidating memories and regulating mood. Additionally, it influences cognitive functions such as learning and concentration. Adequate sleep, typically 7-9 hours per night for adults, helps maintain hormone balance that regulates appetite and growth, contributing to overall health and well-being. Poor or inadequate sleep can lead to issues like fatigue, headaches, and decreased immune function. Therefore, prioritizing good sleep hygiene and creating a restful environment are essential for maintaining overall health.'

In [40]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing exercises, engaging in meditation or mindfulness techniques, doing light stretching or yoga, taking warm baths, and listening to calming music. Additionally, applying peppermint or lavender essential oils can help alleviate headaches. Remember to stay hydrated, maintain a regular sleep schedule, and manage stress through activities you enjoy.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [41]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [42]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [43]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis slightly. Hold for 10 seconds, repeat 8-12 times.\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten your stomach muscles, and raise your shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\nThese gentle stretching and strengthening exercises are recommended to alleviate 

In [44]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and cognitive well-being. During sleep, the body repairs tissues, regenerates cells, and releases hormones that regulate growth and appetite. Sleep occurs in cycles of about 90 minutes, alternating between REM (dreaming and brain activity) and non-REM sleep, with each stage contributing to different restorative processes. Adequate sleep, typically 7-9 hours per night for adults, enhances immune function, improves mood, boosts memory and learning, and helps maintain a healthy weight. Poor or insufficient sleep can lead to a range of health issues, including increased stress, weakened immunity, cognitive impairment, and increased risk of chronic conditions. Therefore, establishing good sleep hygiene and creating a relaxing sleep environment are essential for maintaining overall health.'

In [45]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include practicing deep breathing exercises, progressive muscle relaxation, grounding techniques (such as naming things you see, hear, feel, smell, and taste), taking short walks in nature, and listening to calming music. \n\nFor headaches, natural remedies include staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark and quiet room, gentle massage of the temples and neck, using essential oils like peppermint or lavender, and maintaining a regular sleep schedule.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [47]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [48]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [49]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [50]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [51]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [52]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Do 10-15 repetitions.\n- Partial Crunches: Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Do 8-12 repetitions.\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening abs and tilting pelvis up slightly. Hold for 10 seconds, repeat 8-12 times.\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides.\n\nThese gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes of lower back pain.'

In [54]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical, mental, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, and releases hormones that help regulate growth and appetite. Adequate sleep, typically 7-9 hours per night for adults, is essential for maintaining energy levels, immune function, mental clarity, and emotional well-being. Poor sleep or disruptions can lead to issues such as fatigue, impaired cognitive function, mood disturbances, and increased risk for various health conditions. Therefore, maintaining good sleep hygiene and sufficient quality sleep is fundamental to overall health and wellness.'

In [55]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- For stress:\n  - Deep breathing exercises (e.g., inhale for 4 counts, hold for 4, exhale for 4)\n  - Progressive muscle relaxation\n  - Grounding techniques (identifying things you see, hear, feel, smell, and taste)\n  - Taking short walks in nature\n  - Listening to calming music\n  - Practicing mindfulness and meditation\n  - Engaging in hobbies and leisure activities\n  - Maintaining social connections and support\n\n- For headaches:\n  - Drinking plenty of water to stay hydrated\n  - Applying cold or warm compresses to the head or neck\n  - Resting in a dark, quiet room\n  - Gentle massage of temples and neck\n  - Using essential oils such as peppermint or lavender\n  - Managing stress levels to reduce tension\n  - Maintaining a regular sleep schedule\n\nThese approaches can be effective ways to naturally alleviate stress and headache symptoms.'

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

If sentences are short and highly repetitive, such as in FAQs, semantic chunking may struggle because the embedding distances between consecutive sentences are very similar, resulting in low variance in similarity scores. As a consequence, the algorithm often produces overly large chunks, since everything appears semantically related and there are no clear drops in similarity to trigger splits.

Simply adjusting the threshold is often not sufficient. More precisely, one could lower the breakpoint threshold or use a lower percentile to force splits at smaller semantic changes. However, in FAQ-style content the similarity signal itself is weak, so threshold tuning alone may not yield clean boundaries.

To address this, it is better to incorporate structural cues, such as splitting at question marks, headings, or explicit Q&A markers. Combining semantic similarity with structural boundaries ensures that each question–answer pair forms a coherent chunk, even when semantic distances do not provide a strong separation signal.

---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [56]:
### YOUR CODE HERE
#Creating a golden dataset
### Abstracted SDG
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# Creating LLM and Embeddings Wrapper for Ragas
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

# Create Ragas Testset Generator and generate synthetic dataset
from ragas.testset import TestsetGenerator

# Initialize the Testset Generator with the LLM and Embeddings
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
golden_dataset = generator.generate_with_langchain_docs(raw_docs, testset_size=10)
golden_dataset.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What Chapter 6 talk about in the wellness book...,[The Personal Wellness Guide A Comprehensive R...,"Chapter 6 discusses hydration, emphasizing tha...",single_hop_specifc_query_synthesizer
1,How can the Personal Wellness Guide assist me ...,[The Personal Wellness Guide A Comprehensive R...,The Personal Wellness Guide provides a compreh...,single_hop_specifc_query_synthesizer
2,How does magnesium contribute to sleep and rec...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,The provided context mentions magnesium as a n...,single_hop_specifc_query_synthesizer
3,What role does sleep play in overall health an...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,"Sleep is crucial for physical health, mental w...",single_hop_specifc_query_synthesizer
4,What information is covered in Chapter 17 rega...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 17 discusses supporting digestive heal...,single_hop_specifc_query_synthesizer
5,Could you please explain what is covered in Ch...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 16 focuses on managing headaches natur...,single_hop_specifc_query_synthesizer
6,Chapter 5 and 20 how build healthy habits and ...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,In the context of building healthy habits from...,multi_hop_specific_query_synthesizer
7,H0w do Chapter 6 and Chapter 14 help wellness ...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,Chapter 6 focuses on supporting digestive heal...,multi_hop_specific_query_synthesizer
8,how chapter 8 and 17 relate to building health...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,chapter 8 talks about building healthy habits ...,multi_hop_specific_query_synthesizer
9,How do the principles of habit formation in Ch...,[<1-hop>\n\nPART 5: BUILDING HEALTHY HABITS Ch...,The principles of habit formation in Chapter 1...,multi_hop_specific_query_synthesizer


In [66]:
#Langsmith
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "AIM-Advanced-Retrieval"

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")

# Different retrievers on golden dataset
from ragas import SingleTurnSample
import time

retrieval_chains = {
    "naive": naive_retrieval_chain,
    "bm25": bm25_retrieval_chain,
    "compression": contextual_compression_retrieval_chain,
    "multi_query": multi_query_retrieval_chain,
    "parent_document": parent_document_retrieval_chain,
    "ensemble": ensemble_retrieval_chain,
}

retrieval_results = {name: [] for name in retrieval_chains} # list of samples for each retriever
retrieval_total_time= {} # total time for all queries for each retriever
retrieval_latency= {} # average latency for each query for each retriever
retrieval_total_tokens= {} # total tokens for all queries for each retriever
retrieval_cost= {} # cost for all queries for each retriever
for chain_name, chain in retrieval_chains.items():
  print(f"Starting {chain_name}...")
  retrieval_sum_tokens=0
  retrieval_input_tokens = 0
  retrieval_output_tokens = 0
  retrieval_elapsed = 0
  for gd_row in golden_dataset:
    start = time.time()
    retrieval_response = chain.invoke(
      {"question": gd_row.eval_sample.user_input},
      config={"tags": [chain_name], "metadata": {"retriever": chain_name}}
    )
    end = time.time()
    retrieval_elapsed += (end - start)
    sample = SingleTurnSample(
      user_input=gd_row.eval_sample.user_input,
      reference=gd_row.eval_sample.reference,
      response=retrieval_response["response"].content,
      retrieved_contexts=[context.page_content for context in retrieval_response["context"]]
    )
    retrieval_results[chain_name].append(sample)
    usage = retrieval_response["response"].usage_metadata or {}
    retrieval_sum_tokens += usage.get("total_tokens", 0)
    retrieval_input_tokens += usage.get("input_tokens", 0)
    retrieval_output_tokens += usage.get("output_tokens", 0)
    if chain_name in ["compression", "ensemble"]:
        time.sleep(7)
  retrieval_latency[chain_name]=retrieval_elapsed / len(golden_dataset)
  retrieval_total_time[chain_name]=retrieval_elapsed
  retrieval_total_tokens[chain_name]=retrieval_sum_tokens
  if retrieval_input_tokens == 0 and retrieval_output_tokens == 0 and retrieval_sum_tokens > 0:
    avg_price = (0.10 + 0.40) / 2 / 1e6
    retrieval_cost[chain_name] = retrieval_sum_tokens * avg_price
  else:
    retrieval_cost[chain_name] = (retrieval_input_tokens * 0.10/1e6) + (retrieval_output_tokens * 0.40/1e6)
  

Starting naive...
Starting bm25...
Starting compression...
Starting multi_query...
Starting parent_document...
Starting ensemble...


In [70]:
#Evaluation of retrievers
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))  

from ragas import EvaluationDataset, evaluate, RunConfig
from ragas.metrics import LLMContextPrecisionWithReference, LLMContextRecall, ResponseRelevancy, Faithfulness

custom_run_config = RunConfig(timeout=360)
evaluation_results= {name: [] for name in retrieval_chains}
for chain_name, samples in retrieval_results.items():
    print(f"Starting {chain_name}...")
    evaluation_dataset = EvaluationDataset(samples=samples)
    eval_result = evaluate(
      dataset=evaluation_dataset,
      metrics=[
        LLMContextPrecisionWithReference(), 
        LLMContextRecall(), 
        ResponseRelevancy(), 
        Faithfulness(),
      ],
      llm=evaluator_llm,
      embeddings=generator_embeddings,
      run_config=custom_run_config
    )
    evaluation_results[chain_name]= eval_result

Starting naive...


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

Exception raised in Job[27]: AttributeError('StringIO' object has no attribute 'statements')


Starting bm25...


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

...

Starting parent_document...


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

Starting ensemble...


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

Exception raised in Job[3]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4.1-nano in organization org-ItF4nNeAiZGQC2lBTA4ovFdX on tokens per min (TPM): Limit 200000, Used 200000, Requested 4249. Please try again in 1.274s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})


In [71]:
#Analysis of results
import pandas as pd

rows = []
for name in evaluation_results:
    result = evaluation_results[name]
    scores = result.scores
    row = {k: sum(s[k] for s in scores) / len(scores) for k in scores[0].keys()}
    row["retriever"] = name
    row["total_time"] = retrieval_total_time[name]
    row["latency"] = retrieval_latency[name]
    row["total_tokens"] = retrieval_total_tokens[name]
    row["cost"] = retrieval_cost[name]
    rows.append(row)

df = pd.DataFrame(rows)
df

,llm_context_precision_with_reference,context_recall,answer_relevancy,faithfulness,retriever,total_time,latency,total_tokens,cost
0,0.562930,0.958333,0.844565,NaN,naive,26.630174,2.219181,20244,0.002784
1,0.263889,0.694444,0.535247,0.880661,bm25,14.369782,1.197482,7582,0.001231
2,0.569444,0.777778,0.800698,0.968797,compression,22.922819,1.910235,8267,0.001351
3,0.579956,0.986111,0.732397,NaN,multi_query,36.930649,3.077554,25235,0.003286
4,1.000000,0.937500,0.720290,0.985653,parent_document,24.563068,2.046922,15561,0.002286
5,0.405143,1.000000,0.829466,NaN,ensemble,50.344251,4.195354,39439,0.004692


### Analysis and Conclusion

The Parent Document retriever provides the strongest overall performance because the dataset is highly structured, with clearly defined chapters and thematic sections (e.g., exercise, nutrition, sleep, stress). Since related information is grouped within coherent blocks, preserving the parent context maintains semantic integrity. This explains the very high precision (1.00) and strong recall (0.94), while latency (~2.05s) and cost (~$0.0023 per run) remain moderate, making it the best overall balance between quality and efficiency.

The Naive retriever also performs well because the guide uses explicit headings and clear terminology, making relevant chunks easy to retrieve with simple top-k similarity. Its high recall (0.96) and strong answer relevancy (0.84) show that the dataset is not highly ambiguous. However, its cost (~$0.0028) and latency (~2.22s) are slightly higher than Parent without improving precision.

The Compression retriever is efficient for this dataset, as structured lists and actionable steps allow redundant context to be reduced. Although recall is lower (0.78), it maintains high faithfulness (0.97) with low cost (~$0.00135) and good latency (~1.91s), making it a strong cost-efficient option.

The Multi-Query retriever achieves very high recall (0.99) by capturing semantic variations (e.g., insomnia, stress, hydration). However, because the dataset already uses consistent terminology, the gains do not justify the higher latency (~3.08s) and cost (~$0.0033 per run).

The Ensemble retriever maximizes recall (1.00) but significantly lowers precision (0.41) and has the highest latency (~4.20s) and cost (~$0.0047). In this structured guide, combining retrieval strategies mainly adds redundant context without proportional quality improvements.

Finally, BM25 performs worse because the content is semantically rich and concept-driven rather than keyword-dependent. While it is the fastest (~1.20s) and cheapest (~$0.00123), its precision (0.26) and recall (0.69) are substantially lower.

Overall, this dataset benefits most from retrieval methods that preserve structured context and semantic coherence while maintaining reasonable latency and cost. Among the evaluated approaches, the Parent Document retriever represents the best overall choice, as it achieves the highest precision and strong recall while keeping latency and cost at moderate levels.

![LangSmith Retriever Comparison Dashboard](Langsmith_Retriever_Comparison.png)
The LangSmith monitoring dashboard confirms the numerical evaluation results, showing that Ensemble and Multi-Query incur the highest token usage, cost, and latency, while BM25 remains the fastest and cheapest option. The visual trends in cost, token consumption, and latency align with the reported metrics and support the overall comparative analysis.